# DAOS Filesystem with Tensorflow (Using MNIST)

## Overview

This tutorial shows how to use read and write files on [DAOS Filesystem](https://docs.daos.io/) with TensorFlow, through TensorFlow IO's DAOS file system integration.

A machine running DAOS natively or through a [docker emulator](https://github.com/daos-stack/daos/tree/master/utils/docker) is needed to run this tutorial and/or use the Tensorflow IO DAOS integration. The DAOS Pool and Container used for this tutorial will be created and deleted within this tutorial, where we will be training and testing a simple Neural Network on the MNIST Dataset loaded from the DAOS File System Plugin.

The pool and container id or label are part of the filename uri:
```
dfs://<pool_id>/<cont_id>/<path>
dfs://<pool-label>/cont-label/<path>
```

## Setup and usage

### Install required packages, and restart runtime

In [ ]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

### Create Pool and Container

In [ ]:
!dmg -i pool create -s 500M TEST_POOL
!daos cont create --pool=TEST_POOL --type=POSIX TEST_CONT

Importing the needed libraries

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

Initializing our dfs path

In [ ]:
dfs_url = "dfs://TEST_POOL/TEST_CONT/" # This the path we'll be using to load and access the dataset
pwd = !pwd
posix_url = pwd[0] + "/tests/test_dfs/"

Install Datasets

In [ ]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz -O $(pwd)/tests/test_dfs/train.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz -O $(pwd)/tests/test_dfs/train_labels.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz -O $(pwd)/tests/test_dfs/test.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz -O $(pwd)/tests/test_dfs/test_labels.gz

Copying the Data from the POSIX Filesystem to the DAOS Filesystem under the pool and container we just created

In [ ]:
file_names = ["train.gz", "test.gz", "train_labels.gz", "test_labels.gz"]
for file in file_names:
  tf.io.gfile.copy(posix_url + file, dfs_url + file, True)
    


Checking Our Training Images and Training Labels Exist under the specified pool and container

In [ ]:
images = dfs_url + "train.gz"
labels = dfs_url + "train_labels.gz"
if tf.io.gfile.exists(images) and tf.io.gfile.exists(labels):
  print("True")
else:
  print("False")

Loading MNIST Data from the DFS using tensorflow-io's built in MNIST loading functionality

In [ ]:

d_train = tfio.IODataset.from_mnist(
    images,
    labels
)

Pre-processing and Building a simple Keras Model

In [ ]:
# Shuffle the elements of the dataset.
d_train = d_train.shuffle(buffer_size=1024)

# By default image data is uint8, so convert to float32 using map().
d_train = d_train.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))

# prepare batches the data just like any other tf.data.Dataset
d_train = d_train.batch(32)

# Build the model.
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax),
    ]
)

Compiling the model we just built

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

And finally, training on the dataset for 5 epochs

In [ ]:
history = model.fit(d_train, epochs=15, steps_per_epoch=100)

Plot of Loss vs Epoch during Training

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

Check Test Data is available

In [ ]:
test_images = dfs_url + "test.gz"
test_labels = dfs_url + "test_labels.gz"
if tf.io.gfile.exists(test_images) and tf.io.gfile.exists(test_labels):
  print("True")
else:
  print("False")

Apply same pre-processing and batching on test data

In [ ]:
d_test = tfio.IODataset.from_mnist(
    test_images,
    test_labels,
)

# Shuffle the elements of the dataset.
d_test = d_test.shuffle(buffer_size=1024)

# By default image data is uint8, so convert to float32 using map().
d_test = d_test.map(lambda x, y: (tf.image.convert_image_dtype(x, tf.float32), y))

# prepare batches the data just like any other tf.data.Dataset
d_test = d_test.batch(32)

Evaluate our model on both test and train data

In [ ]:
_, train_acc = model.evaluate(d_train, verbose=0)
_, test_acc = model.evaluate(d_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Prediction Example

In [ ]:
import numpy as np
iterator = iter(d_test)
elem = iterator.get_next()[0][0]
plt.imshow(elem)
prediction = model.predict(np.array([elem]))
result = np.where(prediction[0] == np.amax(prediction[0]))
print("Predicted Value is" ,result[0][0])

### Cleanup

In [ ]:
!dmg -i pool destroy -f TEST_POOL